In [1]:
import json
from rdkit import DataStructs
from rdkit import Chem
import argparse
import sys
import os
import json
import csv
import pandas as pd
import numpy as np
import yaml
import numpy as np
import re
# moo_dir = os.path.abspath(os.path.dirname(os.path.dirname(__file__)))
# print("moo_dir:",moo_dir)
sys.path.append("/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master")
print("sys.path:",sys.path)
from mooseeker.utils import *
from mooseeker.kegg_helper.pykegg import *
import argparse 

sys.path: ['/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/fit_funcs', '', '/opt/ros/lunar/lib/python2.7/dist-packages', '/home/cuisx/anaconda3/envs/pytorch2/lib/python38.zip', '/home/cuisx/anaconda3/envs/pytorch2/lib/python3.8', '/home/cuisx/anaconda3/envs/pytorch2/lib/python3.8/lib-dynload', '/home/cuisx/anaconda3/envs/pytorch2/lib/python3.8/site-packages', '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master']


In [2]:
# 初始化底物池
def init_substrate_pool_from_path(chrom,rxn_dict,all_pool):
    reactions_dict = chrom.get_rxn_dict()
    reactions_list = chrom.get_rxn_list()
    compounds_list = chrom.get_cpd_list()
    # print("reactions_list:",reactions_list)
    # print("compounds_list:",compounds_list)
    # print("equations:")
    # for reaction in reactions_list:
    #     print(rxn_dict[reaction]['equation'])

    substrate_list = [] # 初始化底物列表 需要确保初始化的底物列表中没有重复的

    # 先把反应的所有的反应物都加底物列表中
    for i,reaction in enumerate(reactions_list):
        # 划分出来反应物 split_equation函数在pykegg里面定义了 已经引用了
        lefts,rights = split_equation(reactions_dict[reaction])
        ## 判断遍历的这个底物s 是在lefts里面还是 rights里面
        ## 在哪个里面 哪边就是反应物
        s = compounds_list[i]
        if s in lefts:
            reactants = lefts
        elif s in rights:
            reactants = rights
        else:
            print(s,"没有在方程式里面")
        for r in reactants:
            if r not in substrate_list:
                substrate_list.append(r)
    ## 保留第一个反应中主路径上的化合物  删除主路径上的其余化合物 (因为目标化合物并没有加进去，所以这里应该是1，len-1)
    for i in range(1,len(compounds_list)-1):
        substrate_list.remove(compounds_list[i])

    # 初始化底物池
    substrate_pool = {}
    # 可以通过 chrom 调用all_pool  这里的chrom是 SingleLinkList
    all_pool_cpds = list(all_pool.keys()) # 长度6790 这个键是化合物，对应的值是一个SingleReaction对象 所有的反应对
    for c in substrate_list: # 遍历底物池中的所有 化合物
        if c not in all_pool_cpds: # 如果c不是有效的化合物（这里其实是排除了辅因子和error）
            substrate_pool[c] = None
        else:
            temp = list(all_pool[c]) # 加载反应物c 对应的所有反应 多个SingleReaction 对象
            substrate_pool[c] = temp # ava_pool 是一个字典。键，化合物c。值，化合物c对应的所有反应。
            # 因为是遍历 所以ava_pool中就存储了，当前底物池中，所有化合物对应的所有反应。
    return substrate_pool

# 初始化总反应池 list 和 dict字典 字典中存储反应及01标号 0表示正向反应，1表示逆向反应
def init_total_rxn(chrom,rxn_dict):
    total_rxn_list = []
    total_rxn_dict = {}
    # 这里原本写的是dict  但是必须改为list  因为有可能同一个反应出现了两次
    reactions_list = chrom.get_rxn_list()
    # reactions_dict = chrom.get_rxn_dict()
    compounds_list = chrom.get_cpd_list()
    ## 添加路径上的反应
    for i, key in enumerate(reactions_list):
        total_rxn_dict[key] = rxn_dict[key]

        equation = total_rxn_dict[key]['equation']
        lefts, rights = split_equation(equation)
        s = compounds_list[i]
        ## 判断遍历的这个底物s 是在lefts里面还是 rights里面
        ## 在哪个里面 哪边就是反应物
        if s in lefts:
            total_rxn_dict[key]['direction'] = 0
        elif s in rights:
            total_rxn_dict[key]['direction'] = 1
        else:
            print(s, "没有在方程式里面")
    for rxn in chrom.get_rxn_list():
        total_rxn_list.append(rxn)
    return total_rxn_list, total_rxn_dict

# 判断某个反应的所有反应物是否都在底物池中。也就是判断这个反应能否发生。
def reactants_are_all_in_substrate_pool(reactants,substrate_pool):
    count = 0
    for reactant in reactants:
        if reactant not in substrate_pool.keys():
            break
        else:
            # 反应物在底物池里 count+1
            count+=1
    # 反应物都在底物池中
    if(count==len(reactants)):
        return True
    else:
        return False

# 将反应的生成物添加到底物池中
def add_products_to_substrates_pool(products,need_to_be_added_products,substrate_pool,all_pool_cpds,all_pool):
    for product in products:
        if product not in substrate_pool.keys():
            if product not in need_to_be_added_products.keys():
                if product not in all_pool_cpds: # 如果c不是有效的化合物（这里其实是排除了辅因子和error）
                    need_to_be_added_products[product] = None
                else:
                    temp = list(all_pool[product]) # 加载反应物c 对应的所有反应 多个SingleReaction 对象
                    need_to_be_added_products[product] = temp # ava_pool 是一个字典。键，化合物c。值，化合物c对应的所有反应。
    return need_to_be_added_products

# 将能够发生的反应添加到总反应池
def add_reaction_to_total(reaction,direction,total_rxn_list,total_rxn_dict):
    ## 判断总的列表中是否已经有该反应了
    if reaction['kegg_id'] in total_rxn_list:
#         print(reaction['kegg_id'],"反应已经在总反应里面，不需要再添加了")
        pass
    else:
        # print("添加反应",reaction['kegg_id'])
        total_rxn_list.append(reaction['kegg_id'])
        total_rxn_dict[reaction['kegg_id']] = reaction
        total_rxn_dict[reaction['kegg_id']]['direction'] = direction
    return total_rxn_list,total_rxn_dict

# 底物池中的化合物两两发生反应
def start_having_reactions_in_substrate_pool(substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict):
    ### 遍历底物池
    all_pool_cpds = list(all_pool.keys())
    need_to_be_added_products = {}
    for s in substrate_pool.keys():
#         print("--------------------------------")
#         print("遍历化合物",s,"的所有反应")
        if substrate_pool[s] is None:
            continue
        else:
            singleReactions = substrate_pool[s]# singleReactions是一个列表。包含了某个底物的所有反应
#             print(s,"包含的所有反应个数：",len(singleReactions))
            for singleReaction in singleReactions:
                reaction = singleReaction.reaction  # singleReaction是一个对象。包含了这个反应的所有信息。
                ## 遍历每个反应的每个酶 如果这个酶在酶池里面 那么输出这个反应
                enzymes = reaction['enzyme']
    #             print(reaction)
                # 酶不为空
                if enzymes is not None:
                    for e in enzymes:
                        # 有催化反应的酶
                        if e in enzyme_pool:
                            # 判断底物池中是否存在所有的反应物
                            lefts,rights = split_equation(reaction['equation'])
                            ## 判断遍历的这个底物s 是在lefts里面还是 rights里面
                            ## 在哪个里面 哪边就是反应物
                            if s in lefts:
                                reactants = lefts
                                products = rights
                                direction = 0
                            elif s in rights:
                                reactants = rights
                                products = lefts
                                direction = 1
                            else:
                                print(s,"没有在方程式里面")
                            if reactants_are_all_in_substrate_pool(reactants,substrate_pool):
    #                             print(reaction)
                                total_rxn_list,total_rxn_dict = add_reaction_to_total(reaction,direction,total_rxn_list,total_rxn_dict)
                                # 扩展底物池 上述添加的反应的生成物也要添加到底物池中 直到total_rxn_list的长度不再变化
                                need_to_be_added_products = add_products_to_substrates_pool(products,need_to_be_added_products,substrate_pool,all_pool_cpds,all_pool)
                            break; # 找到一个可以催化的酶就停止
                # 酶为空 也就是说有的反应不需要酶催化
                else:
                    lefts,rights = split_equation(reaction['equation'])
                    if s in lefts:
                        reactants = lefts
                        products = rights
                        direction = 0
                    elif s in rights:
                        reactants = rights
                        products = lefts
                        direction = 1
                    else:
                        print(s,"没有在方程式里面")
                    if reactants_are_all_in_substrate_pool(reactants,substrate_pool):
                        total_rxn_list,total_rxn_dict = add_reaction_to_total(reaction,direction,total_rxn_list,total_rxn_dict)
                        # 扩展底物池 上述添加的反应的生成物也要添加到底物池中 直到total_rxn_list的长度不再变化
                        need_to_be_added_products = add_products_to_substrates_pool(products,need_to_be_added_products,substrate_pool,all_pool_cpds,all_pool)
    return need_to_be_added_products,total_rxn_list,total_rxn_dict,substrate_pool

## 彻底无法继续反应
def react_thoroughly(substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict):
    while True:
        need_to_be_added_products = {}
        need_to_be_added_products,total_rxn_list,total_rxn_dict,substrate_pool = start_having_reactions_in_substrate_pool(substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict)
        if len(need_to_be_added_products)!= 0:
            # print("----------------更新底物池----------------")
            substrate_pool.update(need_to_be_added_products)
        else:
            # print("反应已发生彻底")
            return total_rxn_list,total_rxn_dict,substrate_pool

# 遍历主路径上的所有反应 将生成物添加到底物池中 然后不断反应彻底
def traverse_reactions_in_path(chrom,substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict):
    reactions_dict = chrom.get_rxn_dict()
    reactions_list = chrom.get_rxn_list()
    compounds_list = chrom.get_cpd_list()
    all_pool_cpds = list(all_pool.keys())
    for i,reaction in enumerate(reactions_dict):
        equation = reactions_dict[reaction]
        lefts,rights = split_equation(equation)
        s = compounds_list[i]
        if s in lefts:
            reactants = lefts
            products = rights
        elif s in rights:
            reactants = rights
            products = lefts
        else:
            print(s,"没有在方程式里面")
        # 把生成物添加到底物池中 不要添加重复了
        added_products = {}
        for product in products:
            added_products = add_products_to_substrates_pool(products,added_products,substrate_pool,all_pool_cpds,all_pool)
        total_rxn_list,total_rxn_dict,substrate_pool = react_thoroughly(substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict)
    return total_rxn_list,total_rxn_dict,substrate_pool

In [3]:
# 初始化酶池 为每个反应随机选择酶
def init_enzyme_pool(chrom,rxn_dict):
    reactions_list = chrom.get_rxn_list()
    enzyme_pool = set()
    for i in range(chrom.length()):
        reaction = reactions_list[i]
        ## 以reaction为键值 去all_rxn_dict中查找酶
        enzymes = rxn_dict[reaction]['enzyme']
        # 酶可能是多个，随机选择一个酶，也可能没有
        if enzymes:
            enzyme = enzymes[np.random.randint(len(enzymes))]
            # enzyme = enzymes[0]
            enzyme_pool.add(enzyme)
    # print("选择酶：",enzyme_pool)
    return enzyme_pool


In [4]:
def get_network(cfg, chrom,index):
    #################### 加载反应数据库，化合物数据库，底物池 ####################
    # 获取所有反应
    rxn_dict = read_json(cfg['file_path']['all_rxn_dict'])
    # 获取所有化合物
    cpd_dict = read_json(cfg['file_path']['all_cpd_dict'])
    all_pool = np.load(cfg['file_path']['mypool'], allow_pickle=True).item()

    #################### 针对某个路径chrom 是个SingleLinkList对象 将路径扩展为网络 ####################
    # rxn_list = chrom.get_rxn_list()
    # cpd_list = chrom.get_cpd_list()
    # print("get_network_yield -> rxn_list:",rxn_list)
    # print("get_network_yield -> cpd_list:", cpd_list)
    # 初始化酶池
    print('--- start expanding the path into a network')
#     print('--- (fit_funcs) get enzyme_pool ---')
    enzyme_pool = init_enzyme_pool_list(chrom,rxn_dict)[index]  # 初始化酶池
#     print(enzyme_pool)

    # 初始化底物池
#     print('--- (fit_funcs) get substrate_pool from path---')
    substrate_pool = init_substrate_pool_from_path(chrom,rxn_dict,all_pool)  # 初始化底物池

    # 初始化总反应池
#     print('--- (fit_funcs) init_total_rxn ---')
    total_rxn_list, total_rxn_dict = init_total_rxn(chrom,rxn_dict)

    # 初始底物池反应发生彻底
#     print('--- (fit_funcs) react_thoroughly ---')
    total_rxn_list,total_rxn_dict,substrate_pool = react_thoroughly(substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict)  # 初始底物池中的反应彻底发生完

    # 遍历路径上的反应 不断添加反应所需的反应物 扩展底物池 使底物池反应发生彻底
#     print('--- (fit_funcs) traverse_reactions_in_path ---')
    total_rxn_list,total_rxn_dict,substrate_pool = traverse_reactions_in_path(chrom,substrate_pool,all_pool,enzyme_pool,total_rxn_list,total_rxn_dict)  # 依次把主路径上的反应的生成物添加到底物池中，然后使其彻底反应
#     print(total_rxn_list)

    ####################  计算网络的理论产量值 ####################
    # 先默认返回10 方便调试
    return total_rxn_list, total_rxn_dict, substrate_pool,enzyme_pool


### 构建路径 得到chrom 作为函数的输入

In [5]:
cfg = get_config()
# 获取所有反应
rxn_dict = read_json(cfg['file_path']['all_rxn_dict'])
# 获取所有化合物
cpd_dict = read_json(cfg['file_path']['all_cpd_dict'])
cfg

{'file_path': {'kegg_cofactors': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/kegg/cofactors.csv',
  'all_rxn_dict': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/kegg/all_rxn_dict.json',
  'all_cpd_dict': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/kegg/all_cpd_dict.json',
  'mypool': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/cache/data/MYPOOL_20230914.npy',
  'dG_dict': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/gibbs/dG.json',
  'gibb_concentration': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/gibbs/concentrations.tab',
  'gibb_rxn_ph7_quantity': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-master/mooseeker/data/gibbs/kegg_reactions_CC_ph70_quantity.tsv',
  'yield_dir': '/home/cuisx/products/BDA/pathToNetwork-path-design/mooseeker-mast

In [6]:
# 这个函数位于 init_pool.py 里面 但是因为这里必须要自己构建一个chrom 所以这个函数不可少
def get_Tanimoto(s,p):
    def check_cpd(cpd_name):
        # 1. check cpd name
        pat = re.compile('^C\d{5}')
        res = pat.findall(cpd_name)
        if res != []:
            return True
        else:
            return False
    assert (check_cpd(s) and check_cpd(p))

    # 2. check smile
    if cpd_dict[s]['smile']!=None and cpd_dict[p]['smile']!=None:
        try:
            # 检查smile格式是否存在 存在的话 根据smile格式获得分子的mol文件
            s_smile = cpd_dict[s]['smile']
            s_mol = Chem.MolFromSmiles(s_smile)
            p_smile = cpd_dict[p]['smile']
            p_mol = Chem.MolFromSmiles(p_smile)
            mols = [s_mol, p_mol] # 这里存的就是两个化合物 底物——产物 对
            fps = [Chem.RDKFingerprint(x) for x in mols]
            t = DataStructs.FingerprintSimilarity(fps[0], fps[1]) # 计算相似性
        except Exception as e:
            print(e)
            print("s is %s and p is %s" % (s, p))
            t = 0
    else:
        t = 0
    print('Tanimoto of %s and %s is %s.' %(s, p, str(t)))
    return t

In [7]:
def get_a_chrom(myReactions,myCompounds):
    myRxn_dict = []
    chrom = SingleLinkList()
    for R in myReactions:    
        myRxn_dict.append(rxn_dict[R])
    for i in range(len(myRxn_dict)):
        s = myCompounds[i]
        p = myCompounds[i+1]
        Tanimoto=get_Tanimoto(s,p)
        reaction =  myRxn_dict[i]
        singleReaction = SingleReaction(s,p,reaction,Tanimoto)
        chrom.append(singleReaction)
    return chrom

In [8]:
myReactions = ["R01786","R02740","R04779","R01070","R07159","R01518","R00658","R00200"] # 糖酵解 亚慧师姐论文中的路径
myCompounds = ["C00267","C00668","C05345","C05378","C00118","C00197","C00631","C00074","C00022"]

chrom = get_a_chrom(myReactions,myCompounds) # chrom实际就是一个 SingleLinkList
print(chrom.length())
chrom.travel()

Tanimoto of C00267 and C00668 is 0.6689655172413793.
Tanimoto of C00668 and C05345 is 0.7643564356435644.
Tanimoto of C05345 and C05378 is 0.95.
Tanimoto of C05378 and C00118 is 0.17159763313609466.
Tanimoto of C00118 and C00197 is 0.7354838709677419.
Tanimoto of C00197 and C00631 is 0.43243243243243246.
Tanimoto of C00631 and C00074 is 0.4260089686098655.
Tanimoto of C00074 and C00022 is 0.10404624277456648.
8


[['R01786', 'C00267', 'C00668'],
 ['R02740', 'C00668', 'C05345'],
 ['R04779', 'C05345', 'C05378'],
 ['R01070', 'C05378', 'C00118'],
 ['R07159', 'C00118', 'C00197'],
 ['R01518', 'C00197', 'C00631'],
 ['R00658', 'C00631', 'C00074'],
 ['R00200', 'C00074', 'C00022']]

### 获得酶的所有组合方式   函数改为init_enzyme_pool_list 并且新添加一个递归函数 generate_enzyme_combinations

In [9]:
# 初始化酶池 为每个反应随机选择酶
def init_enzyme_pool_list(chrom,rxn_dict):
    # 反应列表
    reactions_list = chrom.get_rxn_list()
    return generate_enzyme_combinations(reactions_list,rxn_dict, current_combination=[], index=0, result=[])

In [31]:
def generate_enzyme_combinations(reactions_list,rxn_dict, current_combination=[], index=0, result=[]):
    if index == len(reactions_list):
        if current_combination:
            result.append(current_combination.copy())
        return
    reaction = reactions_list[index]
    enzymes = rxn_dict[reaction]['enzyme']

    if enzymes is not None:
        for enzyme in enzymes:
            current_combination.append(enzyme)
            generate_enzyme_combinations(reactions_list,rxn_dict, current_combination, index + 1, result)
            current_combination.pop()
    else:
            generate_enzyme_combinations(reactions_list,rxn_dict, current_combination, index + 1, result)


    return result

In [11]:
enzyme_list = init_enzyme_pool_list(chrom,rxn_dict)
len(enzyme_list)

4

In [12]:
total_rxn_list, total_rxn_dict, substrate_pool,enzyme_pool = get_network(cfg, chrom,0)

--- start expanding the path into a network


In [13]:
print(len(total_rxn_list)) # 最终总的反应
print(total_rxn_list)

20
['R01786', 'R02740', 'R04779', 'R01070', 'R07159', 'R01518', 'R00658', 'R00200', 'R02739', 'R03321', 'R03920', 'R00867', 'R01600', 'R00760', 'R01068', 'R00771', 'R00756', 'R09100', 'R00299', 'R02721']


In [14]:
def get_network_totalReactions(cfg, chrom):
    rxn_list = chrom.get_rxn_list()
    obj_rxn_kegg_id = rxn_list[-1]
    print("get_network_totalReactions -> original-rxn_list:", rxn_list)
    
    rxn_dict = read_json(cfg['file_path']['all_rxn_dict'])
    enzyme_list = init_enzyme_pool_list(chrom,rxn_dict)
    count = 100000 # 设定网络中反应总数最大值100000
    best_enzyme_pool = []
    
    for i in range(len(enzyme_list)):
        total_rxn_list, total_rxn_dict, substrate_pool,enzyme_pool = get_network(cfg, chrom,i)
        if len(total_rxn_list)<count:
            count = len(total_rxn_list)
            best_enzyme_pool = enzyme_pool
    print("best_enzyme_pool:",best_enzyme_pool)
    print("Min-count:",count)
    
    filename = cfg['file_path']['enzyme_rxn_count_dir']
    ## 需要往文件中写入的内容 反应列表 tab键 酶列表 tab 总反应个数
    with open(filename,'a') as file:
        file.write(f"{rxn_list}\t{best_enzyme_pool}\t{count}\n")
        
    return count

In [15]:
get_network_totalReactions(cfg,chrom)

get_network_totalReactions -> original-rxn_list: ['R01786', 'R02740', 'R04779', 'R01070', 'R07159', 'R01518', 'R00658', 'R00200']
--- start expanding the path into a network
--- start expanding the path into a network
--- start expanding the path into a network
--- start expanding the path into a network
best_enzyme_pool: ['2.7.1.2', '5.3.1.9', '2.7.1.11', '4.1.2.13', '1.2.7.6', '5.4.2.11', '4.2.1.11', '2.7.1.40']
Min-count: 17


17

### 特殊情况 网络总反应数>100000

In [27]:
myReactions = ['R02109', 'R02110', 'R02121', 'R02120', 'R00808', 'R00807', 'R10507', 'R00708', 'R00114', 'R05228', 'R00406', 'R01447', 'R13050']
myCompounds = ['C00267', 'C00369', 'C00718', 'C00372', 'C00089', 'C05731', 'C00016', 'C03912', 'C00025', 'C00026', 'C00042', 'C00149', 'C00186', 'C00022']

chrom = get_a_chrom(myReactions,myCompounds) # chrom实际就是一个 SingleLinkList
print(chrom.length())
chrom.travel()

Tanimoto of C00267 and C00369 is 0.485.
Tanimoto of C00369 and C00718 is 0.7823240589198036.
Tanimoto of C00718 and C00372 is 0.5367647058823529.
Tanimoto of C00372 and C00089 is 0.5402985074626866.
Tanimoto of C00089 and C05731 is 0.8044217687074829.
Tanimoto of C05731 and C00016 is 0.2789503707929264.
Tanimoto of C00016 and C03912 is 0.07372175980975029.
Tanimoto of C03912 and C00025 is 0.4425287356321839.
Tanimoto of C00025 and C00026 is 0.3288590604026846.
Tanimoto of C00026 and C00042 is 0.35294117647058826.
Tanimoto of C00042 and C00149 is 0.4827586206896552.
Tanimoto of C00149 and C00186 is 0.41379310344827586.
Tanimoto of C00186 and C00022 is 0.5319148936170213.
13


[['R02109', 'C00267', 'C00369'],
 ['R02110', 'C00369', 'C00718'],
 ['R02121', 'C00718', 'C00372'],
 ['R02120', 'C00372', 'C00089'],
 ['R00808', 'C00089', 'C05731'],
 ['R00807', 'C05731', 'C00016'],
 ['R10507', 'C00016', 'C03912'],
 ['R00708', 'C03912', 'C00025'],
 ['R00114', 'C00025', 'C00026'],
 ['R05228', 'C00026', 'C00042'],
 ['R00406', 'C00042', 'C00149'],
 ['R01447', 'C00149', 'C00186'],
 ['R13050', 'C00186', 'C00022']]

In [32]:
get_network_totalReactions(cfg,chrom)

get_network_totalReactions -> original-rxn_list: ['R02109', 'R02110', 'R02121', 'R02120', 'R00808', 'R00807', 'R10507', 'R00708', 'R00114', 'R05228', 'R00406', 'R01447', 'R13050']
--- start expanding the path into a network
--- start expanding the path into a network
best_enzyme_pool: ['3.2.1.33', '2.4.1.18', '2.4.1.2', '2.4.1.5', '1.1.99.13', '1.1.99.13', '1.5.-.-', '1.2.1.88', '1.4.1.13', '2.8.3.22', '1.1.99.7', '1.3.1.110']
Min-count: 13


13

In [23]:
rxn_dict.get('R02109')

{'bigg_id': None,
 'definition': 'Starch + H2O <=> Amylose + alpha-D-Glucose',
 'enzyme': ['3.2.1.33'],
 'equation': 'C00369 + C00001 <=> C00718 + C00267',
 'kegg_id': 'R02109',
 'name': ['Dextrin', '6-alpha-D-glucanohydrolase'],
 'products': ['C00718', 'C00267'],
 'reactants': ['C00369', 'C00001']}

In [25]:
rxn_dict.get('R05228')

{'bigg_id': None,
 'definition': 'Deacetylcephalosporin C + H2O + 2 Succinate + 2 CO2 <=> Penicillin N + 2 2-Oxoglutarate + 2 Oxygen',
 'enzyme': None,
 'equation': 'C03112 + C00001 + 2 C00042 + 2 C00011 <=> C06564 + 2 C00026 + 2 C00007',
 'kegg_id': 'R05228',
 'name': ['Penicillin-N,2-oxoglutarate:oxygen',
  'oxidoreductase',
  '(ring-expanding)'],
 'products': ['C00026', 'C00007', 'C06564'],
 'reactants': ['C00042', 'C00011', 'C03112', 'C00001']}

### 实现遗传算法中的select函数

In [120]:
import numpy as np

def selection(pop,fitness):
    # 计算适应度总和
    total_fitness = sum(fitness)
    # 计算每个个体被选择的概率，适应度值越小，概率越大
    probabilities = [1 / fit for fit in fitness]
    total_probability = sum(probabilities)
    # 归一化概率，使概率之和为1
    normalized_probabilities = [prob / total_probability for prob in probabilities]

    # 使用numpy进行随机选择
    selected_index = np.random.choice(np.arange(len(fitness)), size=pop_size, replace=True, p=normalized_probabilities)
    pop = [pop[idx] for idx in selected_index]

    return pop


In [112]:
normalized_probabilities

[0.30039337227321494,
 0.15019668613660747,
 0.10013112409107164,
 0.07509834306830374,
 0.060078674454642995,
 0.05006556204553582,
 0.04291333889617356,
 0.03754917153415187,
 0.033377041363690546,
 0.15019668613660747]

In [126]:
pop = [1,2,3,4,5,6,7,8,9,10]
fitness = [1, 2, 3, 4, 5, 6, 7, 8, 9, 2]
pop = selection(pop,fitness)
pop

[1, 1, 4, 6, 9, 1, 2, 10, 1, 5]